# Aprenentage Supervisat

**Exercici 1**

Crea almenys tres models de classificació diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv. Considera si el vol ha arribat tard o no (ArrDelay > 0).

**Exercici 2**

Compara els models de classificació utilitzant la precisió (accuracy), una matriu de confiança i d’altres mètriques més avançades.

**Exercici 3**

Entrena’ls utilitzant els diferents paràmetres que admeten.

**Exercici 4**

Compara el seu rendiment utilitzant l’aproximació traint/test o cross-validation.

**Exercici 5**

Realitza algun procés d’enginyeria de variables per millorar-ne la predicció

**Exercici 6**

No utilitzis la variable DepDelay a l’hora de fer prediccions

In [42]:
import pandas as pd
pd.set_option("display.max_columns", None)

import numpy as np
from scipy import stats 

from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn import metrics

from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler, RobustScaler

# Data

[Airlines Delay: Airline on-time statistics and delay causes](https://www.kaggle.com/giovamata/airlinedelaycauses) 

- Year: 1987-2008
- Month: 1-12
- DayofMonth: 1-31
- DayOfWeek: 1 (Monday) - 7 (Sunday)
- DepTime: departure time (local, hhmm)
- CRSDepTime: scheduled departure time (local, hhmm)
- ArrTime: arrival time (local, hhmm)
- CRSArrTime: scheduled arrival time (local, hhmm)
- UniqueCarrier: unique carrier code
- FlightNum: flight number
- TailNum: plane tail number 
- ActualElapsedTime: flygth time in minutes (Total)
- CRSElapsedTime: scheduled	flygth time in minutes (Total)
- AirTime: time on air in minutes
- ArrDelay:	arrival delay in minutes
- DepDelay: departure delay in minutes
- Origin: origin IATA airport code
- Dest:	destination IATA airport code
- Distance: distance in miles
- TaxiIn: taxi in time, in minutes (movement on ground)
- TaxiOut: taxi out time, in minutes (movement on ground)
- Cancelled: was the flight cancelled?
- CancellationCode:	[reason for cancellation](https://aspmhelp.faa.gov/index/Types_of_Delay.html) (A = carrier, B = weather, C = NAS, D = security)
- Diverted:	1 = yes, 0 = no ("Desviado")
- CarrierDelay:	delayed time due to Carrier in minutes
- WeatherDelay:	delayed time due to Weather in minutes
- NASDelay: delayed time due to NAS in minutes
- SecurityDelay: delayed time due to security in minuts
- LateAircraftDelay: delayed time due to late aircraft in minutes

In [2]:
# Read csv
df_raw = pd.read_csv("./archive/DelayedFlights.csv", index_col = [0])

/Users/luis/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
## Columns and Data types
df_raw.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1936758 entries, 0 to 7009727
Data columns (total 29 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Year               1936758 non-null  int64  
 1   Month              1936758 non-null  int64  
 2   DayofMonth         1936758 non-null  int64  
 3   DayOfWeek          1936758 non-null  int64  
 4   DepTime            1936758 non-null  float64
 5   CRSDepTime         1936758 non-null  int64  
 6   ArrTime            1929648 non-null  float64
 7   CRSArrTime         1936758 non-null  int64  
 8   UniqueCarrier      1936758 non-null  object 
 9   FlightNum          1936758 non-null  int64  
 10  TailNum            1936753 non-null  object 
 11  ActualElapsedTime  1928371 non-null  float64
 12  CRSElapsedTime     1936560 non-null  float64
 13  AirTime            1928371 non-null  float64
 14  ArrDelay           1928371 non-null  float64
 15  DepDelay           1936758 non-n

In [4]:
## Sample
df_raw.sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2626620,2008,5,2,5,913.0,900,1244.0,1209,UA,674,N837UA,151.0,129.0,108.0,35.0,13.0,ORD,LGA,733,4.0,39.0,0,N,0,0.0,0.0,35.0,0.0,0.0
589681,2008,1,2,3,847.0,830,1106.0,1055,DL,1075,N906DE,139.0,145.0,100.0,11.0,17.0,PHL,ATL,665,9.0,30.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2603801,2008,5,30,5,2020.0,1729,2259.0,2010,OO,6684,N751SK,159.0,161.0,139.0,169.0,171.0,SAT,ORD,1041,7.0,13.0,0,N,0,0.0,0.0,169.0,0.0,0.0
1992250,2008,4,10,4,1539.0,1505,1619.0,1555,OO,6610,N956SW,40.0,50.0,24.0,24.0,34.0,DEN,HDN,142,2.0,14.0,0,N,0,0.0,0.0,0.0,0.0,24.0
6212208,2008,11,21,5,1336.0,1327,2034.0,2040,DL,897,N383DN,238.0,253.0,210.0,-6.0,9.0,SNA,ATL,1919,9.0,19.0,0,N,0,NaN,NaN,NaN,NaN,NaN
4954056,2008,9,8,1,956.0,945,1415.0,1408,XE,2156,N21144,139.0,143.0,125.0,7.0,11.0,TUS,IAH,936,7.0,7.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3932961,2008,7,31,4,1804.0,1555,2011.0,1754,DL,821,N675DL,127.0,119.0,91.0,137.0,129.0,DCA,ATL,547,7.0,29.0,0,N,0,0.0,0.0,137.0,0.0,0.0
5812848,2008,10,23,4,2048.0,2035,2322.0,2305,9E,4780,89609E,94.0,90.0,74.0,17.0,13.0,MSP,FWA,488,7.0,13.0,0,N,0,13.0,0.0,4.0,0.0,0.0
6203132,2008,11,14,5,1419.0,1345,1714.0,1621,DL,697,N997DL,175.0,156.0,132.0,53.0,34.0,JFK,ATL,760,10.0,33.0,0,N,0,34.0,0.0,19.0,0.0,0.0
3324501,2008,6,8,7,1335.0,600,1556.0,809,EV,4467,N631AS,81.0,69.0,66.0,467.0,455.0,PFN,ATL,247,10.0,5.0,0,N,0,455.0,0.0,12.0,0.0,0.0


In [5]:
## Drop Duplicates
df_raw.drop_duplicates(inplace = True)

In [6]:
## Null Values %
df_raw.isnull().mean()*100

Year                  0.000000
Month                 0.000000
DayofMonth            0.000000
DayOfWeek             0.000000
DepTime               0.000000
CRSDepTime            0.000000
ArrTime               0.367109
CRSArrTime            0.000000
UniqueCarrier         0.000000
FlightNum             0.000000
TailNum               0.000258
ActualElapsedTime     0.433044
CRSElapsedTime        0.010223
AirTime               0.433044
ArrDelay              0.433044
DepDelay              0.000000
Origin                0.000000
Dest                  0.000000
Distance              0.000000
TaxiIn                0.367109
TaxiOut               0.023493
Cancelled             0.000000
CancellationCode      0.000000
Diverted              0.000000
CarrierDelay         35.588892
WeatherDelay         35.588892
NASDelay             35.588892
SecurityDelay        35.588892
LateAircraftDelay    35.588892
dtype: float64

In [7]:
## Columns with low percentage of nulls (less than 2% in total)
subset = ["ArrTime", "TailNum", "ActualElapsedTime", "CRSElapsedTime", 
          "AirTime", "ArrDelay", "TaxiIn", "TaxiOut"]
## Drop rows with low percentage of nulls
df_raw = df_raw.dropna(subset=subset)

In [8]:
## Transform DepTime and ArrTime to a more consistent notation (hh:mm)
df_raw["DepTime"] = df_raw["DepTime"].astype(int).apply(lambda x: str(x).zfill(4)).apply(lambda x: x[0:2] + ":" + x[2:]) 
df_raw["CRSDepTime"] = df_raw["CRSDepTime"].astype(int).apply(lambda x: str(x).zfill(4)).apply(lambda x: x[0:2] + ":" + x[2:]) 
df_raw["ArrTime"] = df_raw["ArrTime"].astype(int).apply(lambda x: str(x).zfill(4)).apply(lambda x: x[0:2] + ":" + x[2:]) 
df_raw["CRSArrTime"] = df_raw["CRSArrTime"].astype(int).apply(lambda x: str(x).zfill(4)).apply(lambda x: x[0:2] + ":" + x[2:]) 

In [9]:
## Change dtypes
df_raw["FlightNum"] = df_raw["FlightNum"].astype(str)
df_raw["Cancelled"] = df_raw["Cancelled"].astype(str)
df_raw["Diverted"] = df_raw["Diverted"].astype(str)

In [10]:
## Divide into numerical and categorical
df_num = df_raw.select_dtypes(include = ["int64", "float64"])
df_cat = df_raw.select_dtypes(exclude = ["int64", "float64"])

In [11]:
## Describe num
df_num.describe().round(2)

,Year,Month,DayofMonth,DayOfWeek,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1928366.0,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1247484.00,1247484.00,1247484.00,1247484.00,1247484.00
mean,2008.0,6.11,15.75,3.98,133.31,134.20,108.28,42.20,43.09,764.95,6.81,18.22,19.18,3.70,15.02,0.09,25.30
std,0.0,3.48,8.78,2.00,72.06,71.23,68.64,56.78,53.27,573.89,5.27,14.31,43.55,21.49,33.83,2.02,42.05
min,2008.0,1.00,1.00,1.00,14.00,-21.00,0.00,-109.00,6.00,11.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2008.0,3.00,8.00,2.00,80.00,82.00,58.00,9.00,12.00,338.00,4.00,10.00,0.00,0.00,0.00,0.00,0.00
50%,2008.0,6.00,16.00,4.00,116.00,116.00,90.00,24.00,24.00,606.00,6.00,14.00,2.00,0.00,2.00,0.00,8.00
75%,2008.0,9.00,23.00,6.00,165.00,165.00,137.00,56.00,53.00,997.00,8.00,21.00,21.00,0.00,15.00,0.00,33.00
max,2008.0,12.00,31.00,7.00,1114.00,660.00,1091.00,2461.00,2467.00,4962.00,240.00,422.00,2436.00,1352.00,1357.00,392.00,1316.00


In [12]:
## Drop Year
df_raw.drop(columns = "Year", inplace = True)

In [13]:
## Describe cat
df_cat.describe()

,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,Origin,Dest,Cancelled,CancellationCode,Diverted
count,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366
unique,1438,1193,1440,1361,20,7498,5360,303,302,1,1,1
top,18:00,18:00,21:00,19:30,WN,16,N325SW,ATL,ORD,0,N,0
freq,3176,13867,2981,9148,376201,1575,961,131213,108265,1928366,1928366,1928366


In [14]:
## Drop Cancelled, CancellationCode and Diverted
df_raw.drop(columns = ["Cancelled", "CancellationCode", "Diverted"], inplace = True)

In [15]:
## Categorical column with delay > 15 min (1 = Yes, 0 = No)
df_raw["DelayCat"] = df_raw["ArrDelay"].apply(lambda x: 1 if x > 15 else 0)

In [16]:
## Mean Velocity columns in miles/min
df_raw = df_raw[df_raw["AirTime"] != 0]
df_raw["Velocity"] = df_raw["Distance"] / df_raw["AirTime"] 

In [17]:
## Origin-Destination Columns
df_raw["Fligth"] = df_raw["Origin"] + "-" + df_raw["Dest"]

In [18]:
## Save Final Dataframe
df_raw.to_csv("df_clean.csv")

# Exercisi 1

In [19]:
# Read csv
df_clean = pd.read_csv("df_clean.csv", index_col = [0])

/Users/luis/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [20]:
# Sample
df_clean.sample(10)

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DelayCat,Velocity,Fligth
910490,2,15,5,12:39,12:00,15:43,14:50,EV,4195,N761EV,184.0,170.0,151.0,53.0,39.0,SRQ,LGA,1047,9.0,24.0,0.0,39.0,14.0,0.0,0.0,1,6.933775,SRQ-LGA
4526751,8,26,2,18:02,17:55,20:53,21:00,US,753,N161UW,351.0,365.0,322.0,-7.0,7.0,PHL,SEA,2378,6.0,23.0,NaN,NaN,NaN,NaN,NaN,0,7.385093,PHL-SEA
4725268,8,4,1,13:05,12:45,15:17,15:05,AA,638,N3AYAA,132.0,140.0,108.0,12.0,20.0,DFW,MSP,852,8.0,16.0,NaN,NaN,NaN,NaN,NaN,0,7.888889,DFW-MSP
1631339,3,6,4,11:57,10:38,14:50,13:17,NW,1602,N305US,173.0,159.0,151.0,93.0,79.0,DTW,SRQ,1023,4.0,18.0,79.0,0.0,14.0,0.0,0.0,1,6.774834,DTW-SRQ
3011580,6,7,6,12:06,11:30,13:21,12:50,WN,2058,N479WN,75.0,80.0,65.0,31.0,36.0,PHX,BUR,369,2.0,8.0,0.0,0.0,0.0,0.0,31.0,1,5.676923,PHX-BUR
3840913,7,31,4,21:37,20:24,24:00,22:44,UA,321,N801UA,263.0,260.0,237.0,76.0,73.0,ORD,PDX,1739,8.0,18.0,0.0,0.0,3.0,0.0,73.0,1,7.337553,ORD-PDX
3031987,6,13,5,14:10,13:15,15:34,14:25,WN,1429,N222WN,144.0,130.0,118.0,69.0,55.0,PHL,MDW,668,6.0,20.0,36.0,0.0,14.0,0.0,19.0,1,5.661017,PHL-MDW
3952817,7,23,3,22:07,20:25,00:36,22:57,EV,4732,N859AS,149.0,152.0,113.0,99.0,102.0,ATL,HPN,780,16.0,20.0,0.0,0.0,99.0,0.0,0.0,1,6.902655,ATL-HPN
167127,1,28,1,21:00,20:50,21:58,21:52,OH,5450,N915CA,58.0,62.0,45.0,6.0,10.0,CVG,GRR,268,3.0,10.0,NaN,NaN,NaN,NaN,NaN,0,5.955556,CVG-GRR
2737016,5,22,4,15:20,14:55,17:11,16:42,F9,231,N926FR,171.0,167.0,150.0,29.0,25.0,BNA,DEN,1013,8.0,13.0,5.0,0.0,9.0,0.0,15.0,1,6.753333,BNA-DEN


In [21]:
# Variables independientes
x = df_clean[["DepDelay", "Distance", "AirTime"]]
# Variables dependientes
y = df_clean["DelayCat"]

In [22]:
# Split into Test and Train
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33, random_state=1)

In [23]:
# Decision Tree
model_1 = DecisionTreeClassifier(random_state=1).fit(X_train, Y_train)

# Logistic Regression
model_2 = LogisticRegression(random_state=1).fit(X_train, Y_train)

# K-NN
model_3 = KNeighborsClassifier().fit(X_train, Y_train)

# Exercisi 2

In [24]:
# Fitted values
y_1 = model_1.predict(X_test)
y_2 = model_2.predict(X_test)
y_3 = model_3.predict(X_test)

In [25]:
# Accuracy
print("model_1 accuracy:",metrics.accuracy_score(Y_test, y_1))
print("model_2 accuracy:",metrics.accuracy_score(Y_test, y_2))
print("model_3 accuracy:",metrics.accuracy_score(Y_test, y_3))

model_1 accuracy: 0.827118026145619
model_2 accuracy: 0.8593325465656964
model_3 accuracy: 0.8491653296331159


In [26]:
# Porcentaje de unos (no desbalanceado)
Y_test.mean()

0.6295927298898892

El modelo con una accuracy mayor es la regresión logística. La accuracy mide el porcentaje de predicciones correctas. Para datasets desbalanceados esta medida no es fiable ya que devuelve el porcentaje de la medida más observada. 

In [27]:
# Confussion matrix
matrix_1 = metrics.confusion_matrix(Y_test, y_1)
matrix_2 = metrics.confusion_matrix(Y_test, y_2)
matrix_3 = metrics.confusion_matrix(Y_test, y_3)

print("model_1 matrix:",matrix_1 / sum(sum(matrix_1)) * 100)
print("model_2 matrix:",matrix_2 / sum(sum(matrix_2)) * 100)
print("model_3 matrix:",matrix_3 / sum(sum(matrix_3)) * 100)

model_1 matrix: [[30.04530462  6.9954224 ]
 [10.29277499 52.666498  ]]
model_2 matrix: [[31.3412712   5.69945581]
 [ 8.36728953 54.59198346]]
model_3 matrix: [[30.41035013  6.63037688]
 [ 8.45309016 54.50618283]]


La matriz de confusión indica el porcentaje de verdaderos positivos, verdaderos negativos, errores de tipo I y errores de tipo II. La suma de la diagonal devuelve la accuracy.

In [28]:
# Sensitivity
print("model_1 sensitivity:",metrics.recall_score(Y_test, y_1))
print("model_2 sensitivity:",metrics.recall_score(Y_test, y_2))
print("model_3 sensitivity:",metrics.recall_score(Y_test, y_3))

model_1 sensitivity: 0.8365169338594823
model_2 sensitivity: 0.8670999658053099
model_3 sensitivity: 0.8657371701273191


Indica el porcentaje de observaciones positivas correctamente classificadas como positivas.

In [29]:
# Specificity
Sp_1 = matrix_1[0,0] / (matrix_1[0,0] + matrix_1[0,1])
Sp_2 = matrix_2[0,0] / (matrix_2[0,0] + matrix_2[0,1])
Sp_3 = matrix_3[0,0] / (matrix_3[0,0] + matrix_3[0,1])
print("model_1 specificity:",Sp_1)
print("model_2 specificity:",Sp_2)
print("model_3 specificity:",Sp_3)

model_1 specificity: 0.8111424110779256
model_2 specificity: 0.8461300230790116
model_3 specificity: 0.8209976581591094


Indica el porcentaje de observaciones negativas correctamente classificadas como negativas.

# Exercisi 3

In [30]:
# Decision Tree
model_1 = DecisionTreeClassifier(splitter="random", max_depth=3, random_state=1).fit(X_train, Y_train)

# Logistic Regression
model_2 = LogisticRegression(solver="newton-cg", random_state=1).fit(X_train, Y_train)

# K-NN
model_3 = KNeighborsClassifier(n_neighbors=40).fit(X_train, Y_train)

In [31]:
# Fitted values
y_1 = model_1.predict(X_test)
y_2 = model_2.predict(X_test)
y_3 = model_3.predict(X_test)

In [32]:
# Accuracy
print("model_1 accuracy:",metrics.accuracy_score(Y_test, y_1))
print("model_2 accuracy:",metrics.accuracy_score(Y_test, y_2))
print("model_3 accuracy:",metrics.accuracy_score(Y_test, y_3))

model_1 accuracy: 0.7540617795929656
model_2 accuracy: 0.8593325465656964
model_3 accuracy: 0.8635314343004499


En el primer modelo limitamos la profundidad del arbol a tres capas y cambiamos el criterio subdivisión del arbol a aleatorio. La accuracy se ve reducida.

En el segundo modelo cambiamos el algoritmo de optimización por defecto, pero no conseguimos ninguna mejora.

En el tercer modelo aumentamos el radio de vecinos de 5 que hay por defecto a 40, lo que proporciona una mejor accuracy.

# Exercisi 4

In [34]:
# Decision Tree
model_1 = DecisionTreeClassifier(random_state=1)


# Logistic Regression
model_2 = LogisticRegression(random_state=1)

# K-NN
model_3 = KNeighborsClassifier()

In [35]:
# Cross-Validation metrics
scores_1 = cross_val_score(model_1, x, y, cv=5)
scores_2 = cross_val_score(model_2, x, y, cv=5)
scores_3 = cross_val_score(model_3, x, y, cv=5)

print("model_1: %0.2f accuracy with a standard deviation of %0.2f" % (scores_1.mean(), scores_1.std()))
print("model_2: %0.2f accuracy with a standard deviation of %0.2f" % (scores_2.mean(), scores_2.std()))
print("model_3: %0.2f accuracy with a standard deviation of %0.2f" % (scores_3.mean(), scores_3.std()))

model_1: 0.83 accuracy with a standard deviation of 0.00
model_2: 0.86 accuracy with a standard deviation of 0.01
model_3: 0.85 accuracy with a standard deviation of 0.00


Mediante cross-validation podemos optimizar el uso de los datos al entrenar el modelo a la vez que reducimos el riesgo de overfitting. Sin embargo, en esta ocasión la métricas de exactitud no sufren una mejoría demasiado notable.

# Exercisi 5

In [41]:
# Remove outliers from y
z_scores = np.abs(stats.zscore(y))
filtered_entries = z_scores < 3

x = x[filtered_entries]
y = y[filtered_entries]

# Remove outliers from x
z_scores = np.abs(stats.zscore(x))
filtered_entries = (z_scores < 3).all(axis=1)

x = x[filtered_entries]
y = y[filtered_entries]

In [43]:
# Inicializar Scaler
Scaler = StandardScaler()

# Escalar Variables
x.loc[:,:] = Scaler.fit_transform(x.values)

In [44]:
# Split into Test and Train
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33, random_state=1)

In [45]:
# Decision Tree
model_1 = DecisionTreeClassifier(random_state=1).fit(X_train, Y_train)

# Logistic Regression
model_2 = LogisticRegression(random_state=1).fit(X_train, Y_train)

# K-NN
model_3 = KNeighborsClassifier().fit(X_train, Y_train)

In [46]:
# Fitted values
y_1 = model_1.predict(X_test)
y_2 = model_2.predict(X_test)
y_3 = model_3.predict(X_test)

In [47]:
# Accuracy
print("model_1 accuracy:",metrics.accuracy_score(Y_test, y_1))
print("model_2 accuracy:",metrics.accuracy_score(Y_test, y_2))
print("model_3 accuracy:",metrics.accuracy_score(Y_test, y_3))

model_1 accuracy: 0.8253593410675739
model_2 accuracy: 0.8582895970699568
model_3 accuracy: 0.8436115597930771


Tras eliminar los outliers y estandarizar las variables explicativas la exactitud que obtenemos en los modelos es la misma. Esto puede ser debido a que estamos trabajando con un conjunto de datos muy grande dónde el grueso de la población mitiga las posibles desviaciones.

# Exercisi 6

In [48]:
# Remove DepDelay
X_train = X_train[["Distance", "AirTime"]]
X_test = X_test[["Distance", "AirTime"]]

In [49]:
# Decision Tree
model_1 = DecisionTreeClassifier(random_state=1).fit(X_train, Y_train)

# Logistic Regression
model_2 = LogisticRegression(random_state=1).fit(X_train, Y_train)

# K-NN
model_3 = KNeighborsClassifier().fit(X_train, Y_train)

In [50]:
# Fitted values
y_1 = model_1.predict(X_test)
y_2 = model_2.predict(X_test)
y_3 = model_3.predict(X_test)

In [51]:
# Accuracy
print("model_1 accuracy:",metrics.accuracy_score(Y_test, y_1))
print("model_2 accuracy:",metrics.accuracy_score(Y_test, y_2))
print("model_3 accuracy:",metrics.accuracy_score(Y_test, y_3))

model_1 accuracy: 0.623177519315145
model_2 accuracy: 0.6263311125621677
model_3 accuracy: 0.593427041039262


Tras eliminar la variable departure delay se reduce la exactitud de los distintos modelos en un 20% aproximadamente. Aún así, las variables independientes restantes conservan gran parte de la capacidad predictiva de los modelos.